In [38]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#this is a function online for zipcode generation (it can also give us median household income but it's unclear when it has been last updated)
from uszipcode import SearchEngine
search = SearchEngine(simple_zipcode=True)
from uszipcode import Zipcode

#these two don't end up being used yet
from pprint import pprint
import json


source1 = 'Rolling Sales NYC/2018_brooklyn.csv'
source2 = 'Rolling Sales NYC/rollingsales_brooklyn_2019.csv'
df_2018 = pd.read_csv(source1)
df_2019= pd.read_csv(source2)

df_2018 = df_2018.rename(columns={"SALE PRICE\n": "SALE PRICE", "SALE DATE\n": "SALE DATE", "ZIP CODE\n" : "ZIP CODE"})
df_2019 = df_2019.rename(columns={" SALE PRICE ": "SALE PRICE"})

df_2018.dtypes

BOROUGH\n                                  int64
NEIGHBORHOOD\n                            object
BUILDING CLASS CATEGORY\n                 object
TAX CLASS AS OF FINAL ROLL 18/19          object
BLOCK\n                                    int64
LOT\n                                      int64
EASE-MENT\n                              float64
BUILDING CLASS AS OF FINAL ROLL 18/19     object
ADDRESS\n                                 object
APARTMENT NUMBER\n                        object
ZIP CODE                                 float64
RESIDENTIAL UNITS\n                       object
COMMERCIAL UNITS\n                       float64
TOTAL UNITS\n                             object
LAND SQUARE FEET\n                        object
GROSS SQUARE FEET\n                       object
YEAR BUILT\n                             float64
TAX CLASS AT TIME OF SALE\n                int64
BUILDING CLASS AT TIME OF SALE\n          object
SALE PRICE                                object
SALE DATE           

In [39]:
#DF2 is the NYC Rolling Sales, this was pulled from the csv in the first box (source2)
df_2018['SALE PRICE'] = df_2018['SALE PRICE'].str.replace(',','')
df_2018['SALE PRICE'] = df_2018['SALE PRICE'].str.replace('$','').astype(float)
df_2019['SALE PRICE'] = df_2019['SALE PRICE'].str.replace(',','').astype(float)

df_2018 = df_2018.loc[df_2018['SALE PRICE'] > 0]
df_2019 = df_2019.loc[df_2019['SALE PRICE'] > 0]


#Create a boolean mask to sort by the month of September (Airbnb CSV is just September, we can discuss if this step is necessary together)
mask_Sept_2018 = (df_2018['SALE DATE'] > '9/0/2018') & (df_2018['SALE DATE'] <= '9/31/2018')
df_sept = df_2018.loc[mask_Sept_2018]
mask_Oct_2018 = (df_2018['SALE DATE'] > '10/0/2018') & (df_2018['SALE DATE'] <= '10/31/2018')
df_oct = df_2018.loc[mask_Oct_2018]
mask_Nov_2018 = (df_2018['SALE DATE'] > '11/0/2018') & (df_2018['SALE DATE'] <= '11/31/2018')
df_nov = df_2018.loc[mask_Nov_2018]
mask_Dec_2018 = (df_2018['SALE DATE'] > '12/0/2018') & (df_2018['SALE DATE'] <= '12/31/2018')
df_dec = df_2018.loc[mask_Dec_2018]
mask_Jan_2019 = (df_2019['SALE DATE'] > '1/0/2019') & (df_2019['SALE DATE'] <= '1/31/2019')
df_jan = df_2019.loc[mask_Jan_2019]
mask_Feb_2019 = (df_2019['SALE DATE'] > '2/0/2019') & (df_2019['SALE DATE'] <= '2/31/2019')
df_feb = df_2019.loc[mask_Feb_2019]
mask_Mar_2019 = (df_2019['SALE DATE'] > '3/0/2019') & (df_2019['SALE DATE'] <= '3/31/2019')
df_mar = df_2019.loc[mask_Mar_2019]

In [70]:
#Group NYC Sales data by average price by neighborhood
# df_2018_Allzips = df_2018.groupby("ZIP CODE").agg({'SALE PRICE':'mean'}).reset_index()
# df_2019_Allzips = df2_nonzero_sept.groupby("NEIGHBORHOOD").agg({' SALE PRICE ':'count'}).reset_index()

df_sept_allzips = df_sept.groupby("ZIP CODE").agg({"SALE PRICE" : "median"}).reset_index()
df_oct_allzips = df_oct.groupby("ZIP CODE").agg({"SALE PRICE" : "median"}).reset_index()
df_nov_allzips = df_nov.groupby("ZIP CODE").agg({"SALE PRICE" : "median"}).reset_index()
df_dec_allzips = df_dec.groupby("ZIP CODE").agg({"SALE PRICE" : "median"}).reset_index()
df_jan_allzips = df_jan.groupby("ZIP CODE").agg({"SALE PRICE" : "median"}).reset_index()
df_feb_allzips = df_feb.groupby("ZIP CODE").agg({"SALE PRICE" : "median"}).reset_index()
df_mar_allzips = df_mar.groupby("ZIP CODE").agg({"SALE PRICE" : "median"}).reset_index()

In [67]:
dates = [' Sept 2018',' Oct 2018',' Nov 2018',' Dec 2018',' Jan 2019',' Feb 2019',' Mar 2019', '']

df_all_months = pd.merge(df_sept_allzips,df_oct_allzips, on = 'ZIP CODE',how = 'outer', suffixes = (dates[0],dates[1]))
df_all_months = pd.merge(df_all_months,df_nov_allzips, on = 'ZIP CODE',how = 'outer', suffixes = (dates[2],dates[3]))
df_all_months = pd.merge(df_all_months,df_dec_allzips, on = 'ZIP CODE',how = 'outer', suffixes = (dates[2],dates[3]))
df_all_months = pd.merge(df_all_months,df_jan_allzips, on = 'ZIP CODE',how = 'outer', suffixes = (dates[4],dates[5]))
df_all_months = pd.merge(df_all_months,df_feb_allzips, on = 'ZIP CODE',how = 'outer', suffixes = (dates[4],dates[5]))
df_all_months = pd.merge(df_all_months,df_mar_allzips, on = 'ZIP CODE',how = 'outer', suffixes = (dates[6],dates[7]))


In [69]:
Ltrain_zipcodes = [11211,11206,11207,11221,11237,11236,11234,11249]

df_all_months_L = df_all_months[df_all_months['ZIP CODE'].isin(Ltrain_zipcodes)]
df_all_months_L

,ZIP CODE,SALE PRICE Sept 2018,SALE PRICE Oct 2018,SALE PRICE Nov 2018,SALE PRICE Dec 2018,SALE PRICE Jan 2019,SALE PRICE Feb 2019,SALE PRICE
4,11206.0,2.406251e+06,6.847070e+05,9.080516e+05,7.639282e+06,9.458998e+05,8.283777e+05,3.174027e+06
5,11207.0,6.097219e+05,5.432147e+05,6.891468e+05,6.034778e+05,6.611880e+05,6.444369e+05,5.705483e+05
9,11211.0,1.465935e+06,1.556422e+06,8.844303e+05,1.137774e+06,1.207007e+06,7.274168e+06,1.064203e+06
19,11221.0,1.476998e+06,1.043035e+06,1.085451e+06,1.050468e+06,1.045339e+06,1.062166e+06,1.344799e+06
31,11234.0,5.875826e+05,6.838654e+05,5.610347e+05,6.610701e+05,6.011214e+05,6.408765e+05,7.060109e+05
33,11236.0,4.687660e+05,4.880804e+05,5.930058e+05,5.948084e+05,5.528019e+05,5.579517e+05,4.982249e+05
34,11237.0,1.727961e+06,8.816667e+05,2.286857e+06,7.021421e+05,2.562011e+06,1.608167e+06,3.864000e+06
36,11249.0,1.062821e+06,1.554707e+06,9.867249e+05,1.878834e+06,6.344347e+06,2.987418e+06,1.890576e+06
